In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [2]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [3]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([35818725, 1, 53])
Integer dataset size: torch.Size([35818725])


In [4]:
#Get input tensor
def inptarg(i, shuffle, batch_size):
    while True:
        try:
            
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + (seq_length * batch_size)), :, :]
            targets = intdata[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + (seq_length * batch_size))]
    
            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % 75 != 74):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % 75 != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1

            return inp, target
        except:
            continue
        break

In [6]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        output, hidden = self.lstm(inp, hidden)
        output = self.linear(output)
        
        return output, hidden
    

In [10]:
#Set start time
start_time = time.time()

#Define training
def train(epochs):
    #Iterate over desired number of epochs 
    for e in range(epochs):
        
        #Get random order of SMILES molecules (shuffle data)
        shuffle = np.arange(np.shape(data)[0] / seq_length)
        random.shuffle(shuffle)
        
        #Iterate over each molecule in dataset
        for i in range(int(np.shape(data)[0] / seq_length / batch_size)):
            
            #Initialize hidden and cell states
            hidden = model.init_states(num_layers, hidden_size)
            
            #Run on GPU if available
            if cuda:
                hidden = (hidden[0].cuda(), hidden[1].cuda())
        
            #Set initial gradients
            model.zero_grad()
    
            #Set initial loss
            loss = 0 
            
            #Get input and target
            input_data, target_data = inptarg(i, shuffle, batch_size)
            input_data = input_data.float()
            target_data = target_data.long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward()
            optimizer.step()
            
            #Update list of losses
            if (e == i == 0):
                losses[0] = loss.data.item() / seq_length   
            losses.append(loss.data.item() / seq_length)
            
            if (i % 10 == 0):
                torch.save(model.state_dict(), "LSTMtrained128-2.pth")
            
            #Print training info
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Epoch: {0:02d}".format(e + 1) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")
        

In [11]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 128
num_layers = 2
dropout = .2
learning_rate = 0.001
epochs = 2
seq_length = 75
batch_size = 512

#List of losses
losses = [0]

In [12]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [13]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 232629


In [ ]:
#Train
train(epochs)

Loss: 0.052715 | ΔLossTotal: +0.0000 | Epoch: 01 | Iteration: 0001 | Time elapsed: 00h 00 m 19 s
Loss: 0.052359 | ΔLossTotal: -0.0004 | Epoch: 01 | Iteration: 0002 | Time elapsed: 00h 00 m 24 s
Loss: 0.052070 | ΔLossTotal: -0.0006 | Epoch: 01 | Iteration: 0003 | Time elapsed: 00h 00 m 30 s
Loss: 0.051694 | ΔLossTotal: -0.0010 | Epoch: 01 | Iteration: 0004 | Time elapsed: 00h 00 m 35 s
Loss: 0.051222 | ΔLossTotal: -0.0015 | Epoch: 01 | Iteration: 0005 | Time elapsed: 00h 00 m 40 s
Loss: 0.050776 | ΔLossTotal: -0.0019 | Epoch: 01 | Iteration: 0006 | Time elapsed: 00h 00 m 46 s
Loss: 0.050064 | ΔLossTotal: -0.0027 | Epoch: 01 | Iteration: 0007 | Time elapsed: 00h 00 m 51 s
Loss: 0.048875 | ΔLossTotal: -0.0038 | Epoch: 01 | Iteration: 0008 | Time elapsed: 00h 00 m 56 s
Loss: 0.047219 | ΔLossTotal: -0.0055 | Epoch: 01 | Iteration: 0009 | Time elapsed: 00h 01 m 02 s
Loss: 0.045215 | ΔLossTotal: -0.0075 | Epoch: 01 | Iteration: 0010 | Time elapsed: 00h 01 m 07 s
Loss: 0.041936 | ΔLossTotal: -

Loss: 0.025988 | ΔLossTotal: -0.0267 | Epoch: 01 | Iteration: 0086 | Time elapsed: 00h 07 m 58 s
Loss: 0.027614 | ΔLossTotal: -0.0251 | Epoch: 01 | Iteration: 0087 | Time elapsed: 00h 08 m 03 s
Loss: 0.028695 | ΔLossTotal: -0.0240 | Epoch: 01 | Iteration: 0088 | Time elapsed: 00h 08 m 09 s
Loss: 0.024238 | ΔLossTotal: -0.0285 | Epoch: 01 | Iteration: 0089 | Time elapsed: 00h 08 m 14 s
Loss: 0.027007 | ΔLossTotal: -0.0257 | Epoch: 01 | Iteration: 0090 | Time elapsed: 00h 08 m 19 s
Loss: 0.026074 | ΔLossTotal: -0.0266 | Epoch: 01 | Iteration: 0091 | Time elapsed: 00h 08 m 24 s
Loss: 0.029242 | ΔLossTotal: -0.0235 | Epoch: 01 | Iteration: 0092 | Time elapsed: 00h 08 m 29 s
Loss: 0.025816 | ΔLossTotal: -0.0269 | Epoch: 01 | Iteration: 0093 | Time elapsed: 00h 08 m 34 s
Loss: 0.028808 | ΔLossTotal: -0.0239 | Epoch: 01 | Iteration: 0094 | Time elapsed: 00h 08 m 39 s
Loss: 0.025918 | ΔLossTotal: -0.0268 | Epoch: 01 | Iteration: 0095 | Time elapsed: 00h 08 m 44 s
Loss: 0.025511 | ΔLossTotal: -

Loss: 0.022801 | ΔLossTotal: -0.0299 | Epoch: 01 | Iteration: 0171 | Time elapsed: 00h 15 m 25 s
Loss: 0.022815 | ΔLossTotal: -0.0299 | Epoch: 01 | Iteration: 0172 | Time elapsed: 00h 15 m 30 s
Loss: 0.022391 | ΔLossTotal: -0.0303 | Epoch: 01 | Iteration: 0173 | Time elapsed: 00h 15 m 36 s
Loss: 0.022096 | ΔLossTotal: -0.0306 | Epoch: 01 | Iteration: 0174 | Time elapsed: 00h 15 m 41 s
Loss: 0.023153 | ΔLossTotal: -0.0296 | Epoch: 01 | Iteration: 0175 | Time elapsed: 00h 15 m 46 s
Loss: 0.022283 | ΔLossTotal: -0.0304 | Epoch: 01 | Iteration: 0176 | Time elapsed: 00h 15 m 51 s
Loss: 0.020259 | ΔLossTotal: -0.0325 | Epoch: 01 | Iteration: 0177 | Time elapsed: 00h 15 m 57 s
Loss: 0.020393 | ΔLossTotal: -0.0323 | Epoch: 01 | Iteration: 0178 | Time elapsed: 00h 16 m 02 s
Loss: 0.021235 | ΔLossTotal: -0.0315 | Epoch: 01 | Iteration: 0179 | Time elapsed: 00h 16 m 07 s
Loss: 0.022268 | ΔLossTotal: -0.0304 | Epoch: 01 | Iteration: 0180 | Time elapsed: 00h 16 m 12 s
Loss: 0.023398 | ΔLossTotal: -

Loss: 0.017609 | ΔLossTotal: -0.0351 | Epoch: 01 | Iteration: 0256 | Time elapsed: 00h 22 m 58 s
Loss: 0.020004 | ΔLossTotal: -0.0327 | Epoch: 01 | Iteration: 0257 | Time elapsed: 00h 23 m 03 s
Loss: 0.019478 | ΔLossTotal: -0.0332 | Epoch: 01 | Iteration: 0258 | Time elapsed: 00h 23 m 08 s
Loss: 0.021557 | ΔLossTotal: -0.0312 | Epoch: 01 | Iteration: 0259 | Time elapsed: 00h 23 m 14 s
Loss: 0.019305 | ΔLossTotal: -0.0334 | Epoch: 01 | Iteration: 0260 | Time elapsed: 00h 23 m 19 s
Loss: 0.018135 | ΔLossTotal: -0.0346 | Epoch: 01 | Iteration: 0261 | Time elapsed: 00h 23 m 24 s
Loss: 0.019437 | ΔLossTotal: -0.0333 | Epoch: 01 | Iteration: 0262 | Time elapsed: 00h 23 m 30 s
Loss: 0.019055 | ΔLossTotal: -0.0337 | Epoch: 01 | Iteration: 0263 | Time elapsed: 00h 23 m 35 s
Loss: 0.020433 | ΔLossTotal: -0.0323 | Epoch: 01 | Iteration: 0264 | Time elapsed: 00h 23 m 41 s
Loss: 0.019514 | ΔLossTotal: -0.0332 | Epoch: 01 | Iteration: 0265 | Time elapsed: 00h 23 m 46 s
Loss: 0.019193 | ΔLossTotal: -

Loss: 0.016204 | ΔLossTotal: -0.0365 | Epoch: 01 | Iteration: 0341 | Time elapsed: 00h 30 m 32 s
Loss: 0.016538 | ΔLossTotal: -0.0362 | Epoch: 01 | Iteration: 0342 | Time elapsed: 00h 30 m 37 s
Loss: 0.017058 | ΔLossTotal: -0.0357 | Epoch: 01 | Iteration: 0343 | Time elapsed: 00h 30 m 43 s
Loss: 0.015707 | ΔLossTotal: -0.0370 | Epoch: 01 | Iteration: 0344 | Time elapsed: 00h 30 m 48 s
Loss: 0.015558 | ΔLossTotal: -0.0372 | Epoch: 01 | Iteration: 0345 | Time elapsed: 00h 30 m 53 s
Loss: 0.016216 | ΔLossTotal: -0.0365 | Epoch: 01 | Iteration: 0346 | Time elapsed: 00h 30 m 58 s
Loss: 0.015934 | ΔLossTotal: -0.0368 | Epoch: 01 | Iteration: 0347 | Time elapsed: 00h 31 m 04 s
Loss: 0.016480 | ΔLossTotal: -0.0362 | Epoch: 01 | Iteration: 0348 | Time elapsed: 00h 31 m 09 s
Loss: 0.014081 | ΔLossTotal: -0.0386 | Epoch: 01 | Iteration: 0349 | Time elapsed: 00h 31 m 15 s
Loss: 0.015148 | ΔLossTotal: -0.0376 | Epoch: 01 | Iteration: 0350 | Time elapsed: 00h 31 m 20 s
Loss: 0.016721 | ΔLossTotal: -

...

Loss: 0.011443 | ΔLossTotal: -0.0413 | Epoch: 01 | Iteration: 0851 | Time elapsed: 01h 15 m 50 s
Loss: 0.010927 | ΔLossTotal: -0.0418 | Epoch: 01 | Iteration: 0852 | Time elapsed: 01h 15 m 56 s
Loss: 0.011315 | ΔLossTotal: -0.0414 | Epoch: 01 | Iteration: 0853 | Time elapsed: 01h 16 m 01 s
Loss: 0.011020 | ΔLossTotal: -0.0417 | Epoch: 01 | Iteration: 0854 | Time elapsed: 01h 16 m 06 s
Loss: 0.011567 | ΔLossTotal: -0.0411 | Epoch: 01 | Iteration: 0855 | Time elapsed: 01h 16 m 11 s
Loss: 0.011611 | ΔLossTotal: -0.0411 | Epoch: 01 | Iteration: 0856 | Time elapsed: 01h 16 m 17 s
Loss: 0.011946 | ΔLossTotal: -0.0408 | Epoch: 01 | Iteration: 0857 | Time elapsed: 01h 16 m 22 s
Loss: 0.011039 | ΔLossTotal: -0.0417 | Epoch: 01 | Iteration: 0858 | Time elapsed: 01h 16 m 27 s
Loss: 0.011255 | ΔLossTotal: -0.0415 | Epoch: 01 | Iteration: 0859 | Time elapsed: 01h 16 m 33 s
Loss: 0.011253 | ΔLossTotal: -0.0415 | Epoch: 01 | Iteration: 0860 | Time elapsed: 01h 16 m 38 s
Loss: 0.009704 | ΔLossTotal: -

Loss: 0.010652 | ΔLossTotal: -0.0421 | Epoch: 02 | Iteration: 0004 | Time elapsed: 01h 23 m 23 s
Loss: 0.009148 | ΔLossTotal: -0.0436 | Epoch: 02 | Iteration: 0005 | Time elapsed: 01h 23 m 28 s
Loss: 0.010309 | ΔLossTotal: -0.0424 | Epoch: 02 | Iteration: 0006 | Time elapsed: 01h 23 m 34 s
Loss: 0.009076 | ΔLossTotal: -0.0436 | Epoch: 02 | Iteration: 0007 | Time elapsed: 01h 23 m 39 s
Loss: 0.012147 | ΔLossTotal: -0.0406 | Epoch: 02 | Iteration: 0008 | Time elapsed: 01h 23 m 44 s
Loss: 0.009378 | ΔLossTotal: -0.0433 | Epoch: 02 | Iteration: 0009 | Time elapsed: 01h 23 m 50 s
Loss: 0.010996 | ΔLossTotal: -0.0417 | Epoch: 02 | Iteration: 0010 | Time elapsed: 01h 23 m 55 s
Loss: 0.011124 | ΔLossTotal: -0.0416 | Epoch: 02 | Iteration: 0011 | Time elapsed: 01h 24 m 00 s
Loss: 0.011050 | ΔLossTotal: -0.0417 | Epoch: 02 | Iteration: 0012 | Time elapsed: 01h 24 m 05 s
Loss: 0.010706 | ΔLossTotal: -0.0420 | Epoch: 02 | Iteration: 0013 | Time elapsed: 01h 24 m 11 s
Loss: 0.010982 | ΔLossTotal: -

Loss: 0.010138 | ΔLossTotal: -0.0426 | Epoch: 02 | Iteration: 0089 | Time elapsed: 01h 30 m 55 s
Loss: 0.010559 | ΔLossTotal: -0.0422 | Epoch: 02 | Iteration: 0090 | Time elapsed: 01h 31 m 01 s
Loss: 0.010783 | ΔLossTotal: -0.0419 | Epoch: 02 | Iteration: 0091 | Time elapsed: 01h 31 m 06 s
Loss: 0.010125 | ΔLossTotal: -0.0426 | Epoch: 02 | Iteration: 0092 | Time elapsed: 01h 31 m 12 s
Loss: 0.011074 | ΔLossTotal: -0.0416 | Epoch: 02 | Iteration: 0093 | Time elapsed: 01h 31 m 17 s
Loss: 0.010574 | ΔLossTotal: -0.0421 | Epoch: 02 | Iteration: 0094 | Time elapsed: 01h 31 m 22 s
Loss: 0.010577 | ΔLossTotal: -0.0421 | Epoch: 02 | Iteration: 0095 | Time elapsed: 01h 31 m 28 s
Loss: 0.011453 | ΔLossTotal: -0.0413 | Epoch: 02 | Iteration: 0096 | Time elapsed: 01h 31 m 34 s
Loss: 0.010842 | ΔLossTotal: -0.0419 | Epoch: 02 | Iteration: 0097 | Time elapsed: 01h 31 m 39 s
Loss: 0.010489 | ΔLossTotal: -0.0422 | Epoch: 02 | Iteration: 0098 | Time elapsed: 01h 31 m 45 s
Loss: 0.010177 | ΔLossTotal: -

Loss: 0.009208 | ΔLossTotal: -0.0435 | Epoch: 02 | Iteration: 0174 | Time elapsed: 01h 38 m 29 s
Loss: 0.010417 | ΔLossTotal: -0.0423 | Epoch: 02 | Iteration: 0175 | Time elapsed: 01h 38 m 35 s
Loss: 0.010411 | ΔLossTotal: -0.0423 | Epoch: 02 | Iteration: 0176 | Time elapsed: 01h 38 m 40 s
Loss: 0.010598 | ΔLossTotal: -0.0421 | Epoch: 02 | Iteration: 0177 | Time elapsed: 01h 38 m 45 s
Loss: 0.008954 | ΔLossTotal: -0.0438 | Epoch: 02 | Iteration: 0178 | Time elapsed: 01h 38 m 51 s
Loss: 0.010258 | ΔLossTotal: -0.0425 | Epoch: 02 | Iteration: 0179 | Time elapsed: 01h 38 m 56 s
Loss: 0.010492 | ΔLossTotal: -0.0422 | Epoch: 02 | Iteration: 0180 | Time elapsed: 01h 39 m 01 s
Loss: 0.010218 | ΔLossTotal: -0.0425 | Epoch: 02 | Iteration: 0181 | Time elapsed: 01h 39 m 07 s
Loss: 0.008938 | ΔLossTotal: -0.0438 | Epoch: 02 | Iteration: 0182 | Time elapsed: 01h 39 m 12 s
Loss: 0.010151 | ΔLossTotal: -0.0426 | Epoch: 02 | Iteration: 0183 | Time elapsed: 01h 39 m 18 s
Loss: 0.010624 | ΔLossTotal: -

Loss: 0.010343 | ΔLossTotal: -0.0424 | Epoch: 02 | Iteration: 0259 | Time elapsed: 01h 46 m 04 s
Loss: 0.010441 | ΔLossTotal: -0.0423 | Epoch: 02 | Iteration: 0260 | Time elapsed: 01h 46 m 09 s
Loss: 0.011290 | ΔLossTotal: -0.0414 | Epoch: 02 | Iteration: 0261 | Time elapsed: 01h 46 m 14 s
Loss: 0.010322 | ΔLossTotal: -0.0424 | Epoch: 02 | Iteration: 0262 | Time elapsed: 01h 46 m 19 s
Loss: 0.009577 | ΔLossTotal: -0.0431 | Epoch: 02 | Iteration: 0263 | Time elapsed: 01h 46 m 25 s
Loss: 0.009744 | ΔLossTotal: -0.0430 | Epoch: 02 | Iteration: 0264 | Time elapsed: 01h 46 m 30 s
Loss: 0.010243 | ΔLossTotal: -0.0425 | Epoch: 02 | Iteration: 0265 | Time elapsed: 01h 46 m 35 s
Loss: 0.008696 | ΔLossTotal: -0.0440 | Epoch: 02 | Iteration: 0266 | Time elapsed: 01h 46 m 41 s
Loss: 0.008589 | ΔLossTotal: -0.0441 | Epoch: 02 | Iteration: 0267 | Time elapsed: 01h 46 m 46 s
Loss: 0.008504 | ΔLossTotal: -0.0442 | Epoch: 02 | Iteration: 0268 | Time elapsed: 01h 46 m 52 s
Loss: 0.008596 | ΔLossTotal: -

In [23]:
torch.save(model.state_dict(), "LSTMtrained128-2.pth")